# Lab 3.1 : Intro to Building Prompt Templates with LangChain

Welcome to the Lab 3.1. 

In the previous lab, we explored the challenges of prompt engineering; learning how to tweak our wording, choose different models, plus optimize model parameters. Minor changes can significantly enhance the results generated by language models.

In this lab, we will apply our new knowledge to a real-world use case as we continue learning about best practices related to prompt coding. Using the [Watson Machine learning Python SDK](https://ibm.github.io/watson-machine-learning-sdk/) to programmatically interact with watsonx.ai, we will use prompt templating techniques provided by the [LangChain Python library](https://python.langchain.com/) to streamline our interactions with the language model and maximize its potential.

The concept of Prompt Templates provided by LangChain allows you to construct prompt templates that can be easily filled with specific information to generate a wide range of outputs that you can provide to watsonx.ai. You can even leverage prompt templates specific to few-shot prompting, as you will see below.

## Recreating Prompt Builder Prompts Using LangChain Prompt Patterns

### Scenario: Personalized Recommendation for XYZ Retail Company <a id="step3"></a>

XYZ Retail is a popular online retail store in Indonesia that sells a wide range of products, including electronics, clothing, home goods, and more. They have a large customer base and want to provide a personalized shopping experience to enhance customer satisfaction and boost sales.

To achieve that goal, XYZ wants to leverage generative AI to create fact sheets about each of their customers. These fact sheets will summarize relevant information such as customer demographics (name, age, location), and purchase history. These fact sheets will help XYZ Retail's sales team build stronger customer relationships, increase customer satisfaction and drive repeat purchases.


You start by performing prompt engineering in Prompt Lab, and you might test base model output with an initial prompt like this:

![title](./images/prompt_without_example.png)

The model's recommendation is not accurate or useful as the customer Michael Jones had bought toys and games not outdoor activewear. Fortunately you learned in the Prompt Engineering lab that Few Shot Learning can help you obtain better results. 

What happens when we provide a few examples using Prompt Builder to guide the LLM into generating more meaningful recommendations. 

![title](./images/prompt_with_example.png)


Great, the product recommendation for Michael Jones is much better.  However how do you productionize your few shot prompting to generate recommendations for all of XYZ Retail customers? Copy and pasting each customer's info into Prompt Builder would take too long.  

You'll need a programmatic solution.  Maybe you could even generate a large set of examples then use that for Tuning a model in watsonx.ai.  But we're getting ahead of ourselves as you'll learn about building a Prompt Tuning dataset in a later lab.

## 1. Load the required libraries  <a id="step1"></a>

In [1]:
import os

from dotenv import load_dotenv
import pandas as pd
from langchain import PromptTemplate, FewShotPromptTemplate
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams

## 2. Create a Factsheet for each customer using Prompt Patterns  <a id="step2"></a>

### **2.1 What is a Prompt Template?**

The [PromptTemplate class](https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/) in the [LangChain Python library](https://python.langchain.com/docs/get_started/introduction) provides a flexible approach to creating prompts from structured templates.  We will use the PromptTemplate class to simplify creation of our few shot prompts for XYZ Retail.

XYZ Retail has provided you their customer's data in .csv format in Bahasa Indonesia. To generate prompts for each customer, you will need to transform the prompt that you engineered in Prompt Builder into a more useful programmatic format. Using the PromptTemplate class, you can easily substitute customer data to generate one or multiple prompts.

The PromptTemplate class defines a schema where variables to replace are placed inside curly braces "{}". In Python parlance, it's simply using "f-strings" under the hood. These curly braces serve as a placeholder for the actual data that will be substituted into the template.

Let's see how this works in practice.

### **2.2 Creating a simple prompt from a template**

A prompt template can be created using the PromptTemplate class from a string or .txt file. There are [additional PromptTemplate examples](https://api.python.langchain.com/en/latest/prompts/langchain.prompts.prompt.PromptTemplate.html#langchain.prompts.prompt.PromptTemplate) provided in the LangChain documentation.

#### 2.2.1 Prompt Template From String

In [2]:
# template is a string with variable names in curly brackets
# pattern = "input: {nama} {family_name} is {age} and lives in {location}. They bought {purchase_history}"
pattern = "input: {nama} {nama_keluarga} berumur {umur} tahun dan tinggal di daerah {lokasi}. Ia telah membeli {riwayat_transaksi}"

# generate template
prompt_template = PromptTemplate.from_template(pattern)
prompt_template.template

'input: {nama} {nama_keluarga} berumur {umur} tahun dan tinggal di daerah {lokasi}. Ia telah membeli {riwayat_transaksi}'

In [3]:
# now let's provide some values and generate our prompt
# notice how the variables coincide with those we specified in curly brackets
prompt = prompt_template.format(nama="Rudy", 
                                nama_keluarga="Himawari",
                                umur=43,
                                lokasi="Jakarta Selatan, DKI Jakarta",
                                riwayat_transaksi = "bahan pokok, keperluan rumah tangga, dan kebutuhan untuk bepergian")

prompt

'input: Rudy Himawari berumur 43 tahun dan tinggal di daerah Jakarta Selatan, DKI Jakarta. Ia telah membeli bahan pokok, keperluan rumah tangga, dan kebutuhan untuk bepergian'

#### 2.2.2 Prompt Template From File
Prompt patterns can also be stored as a txt file:

In [4]:
# We create a template from a file:
_path_to_file = "./templates/customer_factsheet_lang.txt"

# this time we provide the variable names in a list
example_prompt = PromptTemplate.from_file(_path_to_file,
                                input_variables=["nama", "nama_keluarga", "umur", "kota", "provinsi", 
                                                 "riwayat_transaksi", "rekomendasi_1", "rekomendasi_2"])

print(example_prompt.template)

input: {nama} {nama_keluarga} berumur {umur} tahun dan tinggal di daerah {kota}, {provinsi}. Ia telah membeli {riwayat_transaksi}.
output: Rekomendasi: Barang 1: {rekomendasi_1}, Barang 2: {rekomendasi_2}



Just like in 2.1, we can populate this teamplate from a dictionary containing the values of the input variables. Looping over three examples:

In [5]:
# we can iterate through a list to populate the template

prompt_examples = [
    {
        "nama":"Dina", 
        "nama_keluarga":"Khairunisa", 
        "umur":30, 
        "kota":"Jakarta Selatan", 
        "provinsi":"DKI Jakarta",
        "riwayat_transaksi":"bahan_pokok keperluan_rumah_tangga kebutuhan_untuk_bepergian", 
        "rekomendasi_1":"Buah Keranjang",
        "rekomendasi_2":"Koper berukuran besar"
    },{
        "nama":"Ilham", 
        "nama_keluarga":"Azmiwinata", 
        "umur":40, 
        "kota":"Bandung", 
        "provinsi":"Jawa Barat",
        "riwayat_transaksi":"Buku Barang elektronik perabot rumah tangga", 
        "rekomendasi_1": "Kindle Paperwhite - Perangkat lunak dan Perangkat keras yang dikembangkan oleh Amazon untuk menyimpan dan menampilkan buku elektronik dan media digital lainnya, barang ini sangat cocok untuk pembaca buku yang menginginkan gawai berukuran ringan yang dapat dibawa kemana mana. Produk ini dilengkapi dengan batarai yang tahan lama, sehingga kita dapat membaca buku selama berjam-jam tanpa perlu mengawatirkan dayanya",
        "rekomendasi_2": "Google Home Mini - Merupakan smart speaker yang cocok untuk mengatur perangkat cerdas pada rumah anda hanya dengan suara anda. Anda dapat menggunakannya untuk memutar lagu, mengatur alarm, membaca berita, dan lainnya. Produk ini dapat menjadi salah satu cara untuk tetap terhubung dengan teman dan keluarga."
    },{
        "nama":"Nita", 
        "nama_keluarga":"Simanjuntak", 
        "umur":21, 
        "kota":"Medan", 
        "provinsi":"Sumatera Utara",
        "riwayat_transaksi":"Pakaian Sepatu Kosmetik", 
        "rekomendasi_1":"Baju merek ThenBlank - Baju ini memiliki bahan yang lembut, tipis, dan tidak transparan. Cocok digunakkan untuk keperluan kasual di daerah yang cukup panas. Produknya tersedia dalam berbagai warna, sehingga kita dapat menyesuaikannya dengan style kita",
        "rekomendasi_2":"Sepatu merek Brodo - Dengan bentuknya yang cukup simple seperti sneakers pada umumnya, sepatu ini sangat cocok untuk keperluan kerja maupun tidak. Sepatu ini nyaman dan serbaguna. Sehingga bisa cocok untuk berbagai tipe outfits"
       
    },
    {
        "nama":"Diki", 
        "nama_keluarga":"Simanjuntak", 
        "umur":25, 
        "kota":"Banten", 
        "provinsi":"Banten",
        "riwayat_transaksi":"Pakaian Sepatu Kosmetik", 
        "rekomendasi_1":"Baju merek ThenBlank - Baju ini memiliki bahan yang lembut, tipis, dan tidak transparan. Cocok digunakkan untuk keperluan kasual di daerah yang cukup panas. Produknya tersedia dalam berbagai warna, sehingga kita dapat menyesuaikannya dengan style kita",
        "rekomendasi_2":"Sepatu merek Brodo - Dengan bentuknya yang cukup simple seperti sneakers pada umumnya, sepatu ini sangat cocok untuk keperluan kerja maupun tidak. Sepatu ini nyaman dan serbaguna. Sehingga bisa cocok untuk berbagai tipe outfits"
       
    }
]

for example in prompt_examples: 
    print(example_prompt.format(**example))

input: Dina Khairunisa berumur 30 tahun dan tinggal di daerah Jakarta Selatan, DKI Jakarta. Ia telah membeli bahan_pokok keperluan_rumah_tangga kebutuhan_untuk_bepergian.
output: Rekomendasi: Barang 1: Buah Keranjang, Barang 2: Koper berukuran besar

input: Ilham Azmiwinata berumur 40 tahun dan tinggal di daerah Bandung, Jawa Barat. Ia telah membeli Buku Barang elektronik perabot rumah tangga.
output: Rekomendasi: Barang 1: Kindle Paperwhite - Perangkat lunak dan Perangkat keras yang dikembangkan oleh Amazon untuk menyimpan dan menampilkan buku elektronik dan media digital lainnya, barang ini sangat cocok untuk pembaca buku yang menginginkan gawai berukuran ringan yang dapat dibawa kemana mana. Produk ini dilengkapi dengan batarai yang tahan lama, sehingga kita dapat membaca buku selama berjam-jam tanpa perlu mengawatirkan dayanya, Barang 2: Google Home Mini - Merupakan smart speaker yang cocok untuk mengatur perangkat cerdas pada rumah anda hanya dengan suara anda. Anda dapat mengguna

## 3. Create Prompt Examples based on Customers Factsheet <a id="step3"></a>
The value of PromptTemplate arises when generating a large number of prompts either as examples for bulk evaluation of an engineered prompt or for creation of a Tuning dataset

### 3.1 Create a few shot prompt
We will start by creating a FewShotPromptTemplate object. This class allows to create a prompt made of few repetitions of a PromptTemplate. Details can be found in the [FewShotPromptTemplate class documentation](https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/few_shot_examples)

In [6]:
# Next step create a few shot prompt template

few_shot_examples = prompt_examples[:3]
few_shot_input = prompt_examples[3].copy()
del few_shot_input['rekomendasi_1']
del few_shot_input['rekomendasi_2']

def make_few_shot_prompt(few_shot_examples, few_shot_input):
    """
    Generate a few-shot prompt using the FewShotPromptTemplate class.

    Parameters:
    - few_shot_examples: List of examples to be shown as few-shot examples.
    - few_shot_input: Input for which the prompt will be generated.

    Returns:
    - A string representing the formatted few-shot prompt.
    """
    prompt = FewShotPromptTemplate(
        examples=few_shot_examples, 
        example_prompt=example_prompt, 
        suffix='input: "{nama} {nama_keluarga} berumur {umur} tahun dan tinggal di {kota}, {provinsi}. Riwayat transaksinya berupa {riwayat_transaksi}."\noutput: ', 
        input_variables=["nama", "nama_keluarga", "umur", "kota","provinsi", "riwayat_transaksi"]
    )
    # Return the formatted prompt using the provided input data
    return prompt.format(**few_shot_input)


few_shot_prompt = make_few_shot_prompt(few_shot_examples, few_shot_input)
print(few_shot_prompt)

input: Dina Khairunisa berumur 30 tahun dan tinggal di daerah Jakarta Selatan, DKI Jakarta. Ia telah membeli bahan_pokok keperluan_rumah_tangga kebutuhan_untuk_bepergian.
output: Rekomendasi: Barang 1: Buah Keranjang, Barang 2: Koper berukuran besar


input: Ilham Azmiwinata berumur 40 tahun dan tinggal di daerah Bandung, Jawa Barat. Ia telah membeli Buku Barang elektronik perabot rumah tangga.
output: Rekomendasi: Barang 1: Kindle Paperwhite - Perangkat lunak dan Perangkat keras yang dikembangkan oleh Amazon untuk menyimpan dan menampilkan buku elektronik dan media digital lainnya, barang ini sangat cocok untuk pembaca buku yang menginginkan gawai berukuran ringan yang dapat dibawa kemana mana. Produk ini dilengkapi dengan batarai yang tahan lama, sehingga kita dapat membaca buku selama berjam-jam tanpa perlu mengawatirkan dayanya, Barang 2: Google Home Mini - Merupakan smart speaker yang cocok untuk mengatur perangkat cerdas pada rumah anda hanya dengan suara anda. Anda dapat menggun

### 3.2 Bulk Creation of Prompts
Using the FewShotPromptTemplate class, we can now create a function that generates a list a few shot prompts populating them iteratively from values directy extracted from a notebook.

We can choose how many single prompts are include in one few shot prompt. The output of the function is a list of few shot prompts. 

In [7]:
# Specify the path to the CSV file containing the data
csv_file_path = "./data/customer_factsheet.csv"

def sub_all_from_csv(csv_file_path, n_prompt_examples=2):
    """
    Generates a list of few-shot prompts using the FewShotPromptTemplate class. 
    The prompts are populated iteratively from values extracted from a CSV file.

    Parameters:
    - csv_file_path: The path to the CSV file.
    - n_prompt_examples: The number of examples included in one few-shot prompt.

    Returns:
    - list_of_prompts: A list of few-shot prompts.
    """

    df = pd.read_csv(csv_file_path)
    examples = [example for _, example in df.transpose().to_dict().items()]

    i=0
    list_of_prompts = []

    while i < len(df):
        few_shot_examples = examples[i:i+n_prompt_examples]
        few_shot_input = examples[i+n_prompt_examples].copy()
        del few_shot_input['rekomendasi_1']
        del few_shot_input['rekomendasi_2']

        list_of_prompts.append(make_few_shot_prompt(few_shot_examples, few_shot_input))
        
        i = i+n_prompt_examples +1

    # Return the list of few-shot prompts
    return list_of_prompts

### 3.2 Additional Examples
You can explore [additional examples using the PromptTemplate](https://api.python.langchain.com/en/latest/prompts/langchain.prompts.prompt.PromptTemplate.html#langchain.prompts.prompt.PromptTemplate)

## 4. Prompt evaluation and few shot learning from bulk created prompts <a id="step4"></a>
In the prior examples, you created a "2-shot learning" prompt.  I.e. there were three inputs but only two complete outputs.  By using a larger dataset this way, you can perform bulk testing of your prompt.

E.g. two of your data sample are used to train while the "output" of the 3rd can be compared against the model output to ensure your prompt is performing as expected.  You can now execute these few shot prompts to see how well our engineered prompt works across numerous examples

### 4.1 Import Watsonx.ai access credentials and load model
Make sure you copied the .env file that you created earlier into the same directory as this notebook

In [8]:
load_dotenv()
api_key = os.getenv("API_KEY", None)
ibm_cloud_url = os.getenv("IBM_CLOUD_URL", 'https://us-south.ml.cloud.ibm.com')
project_id = os.getenv("PROJECT_ID", None)
if api_key is None or ibm_cloud_url is None or project_id is None:
    print("Ensure you copied the .env file that you created earlier into the same directory as this notebook")
else:
    creds = {
        "url": ibm_cloud_url,
        "apikey": api_key
    }


model_params = {
    GenParams.DECODING_METHOD: "greedy",
    GenParams.MIN_NEW_TOKENS: 50,
    GenParams.MAX_NEW_TOKENS: 300,
    GenParams.STOP_SEQUENCES: ["\n\n"]
}

# Instantiate a model proxy object to send your requests
# As we know not every model can works very good with every task, for some taskes that need to use Bahasa Indonesia, LLAMA2 can work very good. 
model = Model(
    model_id="meta-llama/llama-2-70b-chat",
    params=model_params,
    credentials=creds,
    project_id=project_id)

### 4.2 Send prompts to Watsonx.ai

In [9]:
list_of_prompts = sub_all_from_csv(csv_file_path)
responses = [model.generate_text(prompt) for prompt in list_of_prompts]
for i, response in enumerate(responses):
    lines = str(list_of_prompts[i]).strip().split("\n")
    user_description = str(lines[8])
    print(f"\n{user_description}")
    print(f"\noutput: {response}")
    print("---------------------")


input: "Nita Simanjuntak berumur 25 tahun dan tinggal di Medan, Sumatera Utara. Riwayat transaksinya berupa Pakaian, Sepatu, dan Kosmetik."

output:  Rekomendasi: Barang 1: Sneakers Nike Air Force 1 - Sepatu ini sangat cocok untuk pengguna yang menginginkan sepatu yang stylish dan nyaman untuk dipakai sehari-hari. Barang ini memiliki desain yang elegan dan tahan lama, serta dilengkapi dengan teknologi Nike yang terkini untuk meningkatkan kenyamanan dan kestabilan kaki. Barang 2: MacBook Air - Laptop ini sangat cocok untuk pengguna yang menginginkan perangkat lunak yang ringan dan mudah digunakan untuk kebutuhan kerja dan perkuliahan. MacBook Air memiliki prosesor Intel Core i5 yang cepat dan RAM 8 GB yang dapat meningkatkan kemampuan multitasking dan meningkatkan efisiensi kerja.


---------------------

input: "Kintan Arsadian berumur 22 tahun dan tinggal di Surabaya, Jawa Timur. Riwayat transaksinya berupa Makeup, Skincare, dan Pakaian."

output:  Rekomendasi: Barang 1: Makeup - Kin

### Few shot prompt analysis


Pendekatan yang kita lakukan adalah memberikan beberapa contoh input (Few Shot), sehingga model dapat memberikan respon.

Rekomendasi yang diberikan sudah cukup sesuai dengan karakteristik dari para user, namun kita masih dapat melakukan beberapa cara untuk mendapatkan hasil serupa karena masih terdapat beberapa teknik yang bisa kita lakukan, seperti memberikan prompt instruction dan menentukan jumlah sample yang akan kita gunakan sebagai shot. 

### 4.3 Additional Technique on Few Shot Prompt Analysis

In [11]:
def new_shot(values):
    '''Prompt yang kita gunakan sebelumnya, kebanyakan mengambil data dari factsheet yang telah dibuat pada directory data/customer_factsheet.csv 
    function ini akan membantu kita untuk membuat data baru yang kita bisa gunakan sebagai analisis atau input'''
    keys = ['nama', 'nama_keluarga', 'umur', 'kota', 'provinsi', 'riwayat_transaksi', 'rekomendasi_1', 'rekomendasi_2']
    prompt_shot = dict(zip(keys, values))
    return prompt_shot

new_values = ['Dony', 'Setiawan', 50, 'Jakarta', 'DKI Jakarta', 'Games, Komputer, Tumblr']
new_shot_input = new_shot(new_values)

In [28]:
# Function
csv_file_path = "./data/customer_factsheet.csv"
df = pd.read_csv(csv_file_path)

def few_shot(name_to_test, number_of_examples, no_extra_sample=True, new_shot=None):
    '''
    Fungsi ini berguna untuk membuat data factsheet kita berada dalam format yang dapat kita jadikan input dalam model Generative AI kita. 
    name_to_test = variable yang diisikan nama user yang ingin kita test rekomendasinya dan telah berada dalam fact sheet
    number_of_example = Jumlah few shot test yang ingin kita gunakan
    no_extra_sample = True -> kita menggunakan data yang telah berada pada fact sheet
                      False -> menggunakan data baru dari function "few_shot" 
    new_shot = Variable data baru yang ingin kita panggil apabila no_extra_sample = False 
    '''

    # Filter the DataFrame to exclude specified names
    df_filtered = df[~df['nama'].isin(name_to_test)]
    few_shot_train = [example for _, example in df_filtered.transpose().to_dict().items()]

    if no_extra_sample==True:
        few_shot_test = df[df['nama'].isin(name_to_test)].iloc[0].to_dict()
        few_shot_test.pop('rekomendasi_1', None)
        few_shot_test.pop('rekomendasi_2', None)
    else:
        few_shot_input = new_shot

    return make_few_shot_prompt(few_shot_train[:number_of_examples - 1], few_shot_test)
few_shot_model = few_shot(name_to_test=['Dikta'], number_of_examples=3, no_extra_sample=True, new_shot = new_shot_input)

In [36]:
# Generative AI Model
'''Untuk bisa mendapatkan model dengan hasil yang baik, cobalah juga untuk melakukan adjustment terhadap model_parameters.'''
model_parameters = {
    GenParams.DECODING_METHOD: "greedy",
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 300,
    GenParams.STOP_SEQUENCES: ["\n\n"]
}

llama2 = "meta-llama/llama-2-70b-chat"

model = Model(model_id=llama2,params=model_parameters,credentials=creds,project_id=project_id)

def prompt_ID(shot_list):
    ''' Fungsi ini adalah fungsi generative AI yang akan digunakan untuk mengeluarkan atau menghasilkan rekomendasi produk yang sesuai dengan fact sheet yang ada.
    Perbedaan utama dari fungsi ini adalah:
    1. Jumlah Few Shot atau Sample yang dapat digunakan
    2. Instruction, untuk memberikan clarity atau penjelasan terhadap model Generative AI yang akan kita gunakan
    Dibawah ini, kita menggunakan prompt template untuk model LLAMA2
    '''
    segments = shot_list.split("\n\ninput:")
    sample_shot_list = "\n\ninput:".join(segments[:-1])
    test_shot = "input:" + segments[-1]
    prompt_en_id = ""
    prompt_en_id +=  "<s>[INST] <<SYS>>\n"\
                    "Kamu adalah model AI yang bertugas untuk memberikan rekomendasi terkait barang dan brand tertentu bagi pengguna kita.\n"\
                    "Konsumen kita memiliki beberapa karakteristik seperti nama, nama keluarga, kota, dan riwayat transaksi.\n"\
                    "Pastikan kamu menuliskan rekomendasi yang relevan dan sesuai dengan riwayat transaksi mereka serta ikuti pola-pola transaksi seperti pada contoh di dalam tanda ''' .\n"\
                    "Ingat: Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.\n"\
                    "Please ensure that your responses are socially unbiased and positive in nature.\n"\
                    "Berikut adalah contoh yang dapat anda ikuti:\n"\
                    "'''\n"\
                    f"{sample_shot_list}\n"\
                    "'''\n"\
                    "<</SYS>>\n"\
                    "Berikan dua rekomendasi barang serta gunakan Bahasa Indonesia!\n"\
                    "[/INST]"\
                    f"{test_shot}"
    return prompt_en_id

# Call prompt_definitions to get the full prompt
prompts = prompt_ID(few_shot_model)
responses = model.generate_text(prompts)
print(prompts,responses)
# print(prompts,responses)

<s>[INST] <<SYS>>
Kamu adalah model AI yang bertugas untuk memberikan rekomendasi terkait barang dan brand tertentu bagi pengguna kita.
Konsumen kita memiliki beberapa karakteristik seperti nama, nama keluarga, kota, dan riwayat transaksi.
Pastikan kamu menuliskan rekomendasi yang relevan dan sesuai dengan riwayat transaksi mereka serta ikuti pola-pola transaksi seperti pada contoh di dalam tanda ''' .
Ingat: Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature.
Berikut adalah contoh yang dapat anda ikuti:
'''
input: Ilham Azmiwinata berumur 40 tahun dan tinggal di daerah Bandung, Jawa Barat. Ia telah membeli Buku, Barang elektronik, dan perabot rumah tangga.
output: Rekomendasi: Barang 1: Kindle Paperwhite - Perangkat lunak dan Perangkat keras yang dikembangkan oleh Amazon untuk menyimpan dan m

## 5. Congratulations
Congratulations on completing the lab and exploring the fascinating world of bulk creation of Few Shot Prompts using PromptTemplate! 

Through the practical use case of generating personalized product recommendations, you have witnessed the power of tailoring prompts to individual customer profiles. By incorporating customer-specific details and programmatically generating bulk examples, you can fine-tune the model for your specific use case, resulting in more accurate and tailored outputs. 

Continuously iterating and refining your prompts based on these examples will unlock the full potential of language models and enhance their performance across various domains. Keep experimenting and leveraging prompt engineering techniques to optimize your interactions with language models and drive impactful results in your projects.